# AIMO Prize 3 - OOP Baseline Submission
This notebook demonstrates how to use the Object-Oriented Baseline designed for stability across Local and Kaggle environments.

In [ ]:
import sys
import os
sys.path.append('/kaggle/input/aimo-modules')
sys.path.append(os.path.join(os.getcwd(), 'src'))
try:
    from kaggle_baseline import CompetitionConfig, AIMSolver, MockLLM
    print("Successfully imported OOP Baseline modules.")
except ImportError as e:
    print(f"Import Failed: {e}. Make sure the script is attached.")

In [ ]:
config = CompetitionConfig()
llm = MockLLM()
solver = AIMSolver(config, llm)
print("Solver initialized.")

## Submission Loop

In [ ]:
import aimo_3_gateway as aimo
try:
    env = aimo.make_env()
    iter_test = env.iter_test()
    for test, sample_submission in iter_test:
        for index, row in test.iterrows():
            result = solver.solve(row['problem'])
            sample_submission.loc[index, 'answer'] = result
        env.predict(sample_submission)
    print("Submission loop complete.")
except ImportError:
    print("Submission loop skipped (Local Mode).")